In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Download packages

In [2]:
!pip uninstall -y tensorflow
!pip install -q tf-nightly-gpu
!pip install -q tensorflow-model-optimization==0.5
!pip install tensorflow==2.4.0

import tempfile
import os
import numpy as np

import tensorflow as tf

from tensorflow import keras

import tensorflow_model_optimization as tfmot

%load_ext tensorboard

Uninstalling tensorflow-2.4.0:
  Successfully uninstalled tensorflow-2.4.0
ERROR: fancyimpute 0.4.3 requires tensorflow, which is not installed.
  Using cached https://files.pythonhosted.org/packages/94/0a/012cc33c643d844433d13001dd1db179e7020b05ddbbd0a9dc86c38a8efa/tensorflow-2.4.0-cp37-cp37m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d6/84/759f5dd23fec8ba71952d97bcc7e2c9d7d63bdc582421f3cd4be845f0c98/gast-0.3.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/06/54/1c8be62beafe7fb1548d2968e518ca040556b46b0275399d4f3186c56d79/grpcio-1.32.0-cp37-cp37m-manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/3f/c0/abde58b837e066bca19a3f7332d9d0493521d7dd6b48248451a9e3fe2214/h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl
ERROR: tf-nightly-gpu 2.5.0.dev20210318 has requirement gast==0.4.0, but you'll have gast 0.3.3 which is incompatible.
ERROR: tf-nightly-gpu 2.5.0.dev20210318 has requirement grpcio~=1.34

## Download dataset

In [3]:
import tensorflow_datasets as tfds
import tensorflow as tf

BATCH_SIZE = 32
IMAGE_SIZE = 224

def format_image(image, label):
    image = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE)) / 255.0
    return  image, label

(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    data_dir='/content/dataset/',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    shuffle_files=True, 
    as_supervised=True,
)

# tf.data.experimental.cardinality(raw_train_dataset).numpy()
num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes
print("Numbers of images: ", num_examples)
print("Numbers of classes: ", num_classes)

train_batches = raw_train.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = raw_validation.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = raw_test.map(format_image).batch(1)

Numbers of images:  23262
Numbers of classes:  2


## Build a Keras model

In [4]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    # Output neuron
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
print(model.summary())

from tensorflow.keras.optimizers import RMSprop
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        3

## Training our model

We will train our model within 3 epoches

In [5]:
history = model.fit(
      train_batches,  
      epochs=3,
      validation_data=validation_batches)

Epoch 1/3
582/582 [==============================] - 37s 49ms/step - loss: 0.6751 - acc: 0.5959 - val_loss: 0.5292 - val_acc: 0.7532
Epoch 2/3
582/582 [==============================] - 31s 47ms/step - loss: 0.5128 - acc: 0.7547 - val_loss: 0.5003 - val_acc: 0.7554
Epoch 3/3
582/582 [==============================] - 31s 47ms/step - loss: 0.4398 - acc: 0.7977 - val_loss: 0.4631 - val_acc: 0.7966


##Fine-tune pre-trained model with pruning

In [6]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
epochs = 2
end_step = np.ceil(num_examples*0.8 / BATCH_SIZE).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)
# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_conv2d ( (None, 222, 222, 16)      882       
_________________________________________________________________
prune_low_magnitude_max_pool (None, 111, 111, 16)      1         
_________________________________________________________________
prune_low_magnitude_conv2d_1 (None, 109, 109, 32)      9250      
_________________________________________________________________
prune_low_magnitude_max_pool (None, 54, 54, 32)        1         
_________________________________________________________________
prune_low_magnitude_conv2d_2 (None, 52, 52, 64)        36930     
_________________________________________________________________
prune_low_magnitude_max_pool (None, 26, 26, 64)        1         
_________________________________________________________________
prune_low_magnitude_conv2d_3 (None, 24, 24, 64)        7

Training our pruned model

In [7]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_batches,
                  batch_size=BATCH_SIZE, epochs=epochs, validation_data=validation_batches,
                  callbacks=callbacks)


Epoch 1/2
  6/582 [..............................] - ETA: 2:27 - loss: 0.4952 - accuracy: 0.7604WARNING:tensorflow:Callback method `on_train_batch_begin` is slow compared to the batch time (batch time: 0.0299s vs `on_train_batch_begin` time: 0.1047s). Check your callbacks.


582/582 [==============================] - 40s 57ms/step - loss: 0.3889 - accuracy: 0.8271 - val_loss: 0.3826 - val_accuracy: 0.8263
Epoch 2/2
582/582 [==============================] - 35s 54ms/step - loss: 0.3448 - accuracy: 0.8500 - val_loss: 0.4064 - val_accuracy: 0.8267


##Evaluate our models

Evaluate and save our models

In [8]:
_, baseline_model_accuracy = model.evaluate(test_batches, verbose=0)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

_, model_for_pruning_accuracy = model_for_pruning.evaluate(test_batches, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)


Saved baseline model to: /tmp/tmpeb7ilfgh.h5
Baseline test accuracy: 0.845227837562561
Pruned test accuracy: 0.845227837562561


## Use TFlite to convert our model

In [9]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)


Saved pruned Keras model to: /tmp/tmp9al9srji.h5
INFO:tensorflow:Assets written to: /tmp/tmp3x9_1dgv/assets


INFO:tensorflow:Assets written to: /tmp/tmp3x9_1dgv/assets


Saved pruned TFLite model to: /tmp/tmphupiwp5y.tflite



Applying a standard compression algorithm is necessary since the serialized weight matrices are the same size as they were before pruning. However, pruning makes most of the weights zeros, which is added redundancy that algorithms can utilize to further compress the model.

In [10]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)


In [11]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

Size of gzipped baseline Keras model: 2926328.00 bytes
Size of gzipped pruned Keras model: 2926328.00 bytes
Size of gzipped pruned TFlite model: 2871188.00 bytes


## Excercice

What can we see in term of the size and the accuracy of models ? *

Try to create a smaller model from combining pruning and quantization

## Clean Up

Before running the next exercise, run the following cell to terminate the kernel and free memory resources:

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)